## Creates PyG object from an HRG

In [ ]:
import glob
from sexpdata import loads
from os.path import basename
from tqdm import tqdm 
import json

In [ ]:
def read_and_parse_hrg(pth: str):
    hrg = []
    with open(pth, "r", encoding ='ISO-8859-1') as f:
        for line in f:
            hrg.append(line.strip())
    hrg = " ".join(hrg)
    parsed_hrg = loads(hrg)
    return parsed_hrg

In [ ]:
pth = '/usr0/home/amadaan/data/audio/LJSpeech-1.1/TTS/hrg/LJ016-0056.hrg'

In [ ]:
parsed_hrg = read_and_parse_hrg(pth)

In [ ]:
parsed_hrg

In [ ]:
def nodes_edges_from_hrg(hrg_path):
    parsed_hrg = read_and_parse_hrg(hrg_path)
    res = []
    for root in parsed_hrg:
        word_tree = make_tree(root)
        res.append({
            "word": word_tree[0],
            "daughters": word_tree[1:]
        })
    return res
        
def make_tree(root, rep=None, depth=0):
    if depth == 2:
        return
    if rep is None:
        rep = []
    if parse_avl(root) != "syl":
        rep.append(parse_avl(root))
    for daughter in root[1:]:
        make_tree(daughter, rep, depth=depth+1)
        if parse_avl(root) == "syl":
            rep.append(parse_syll(daughter))
        elif parse_avl(daughter) != "syl":
            rep.append(parse_avl(daughter))

    return rep


In [ ]:
def nodes_edges_from_hrg(hrg_path):
    parsed_hrg = read_and_parse_hrg(hrg_path)
    res = []
    for root in parsed_hrg:
        word_tree = make_tree_two_levels(root)
        res.append({
            "word": word_tree[0],
            "daughters": word_tree[1:]
        })
    return res
        
def make_tree_two_levels(root, rep=None, depth=0):
    if depth == 2:
        return
    if rep is None:
        rep = []
    if parse_avl(root) != "syl":
        rep.append(parse_avl(root))
    if parse_avl(root) == "syl":
        tmp = []
        for daughter in root[1:]:
            tmp.append(parse_syll(daughter))
        rep.append(tmp)
    else:
        for daughter in root[1:]:
            make_tree_two_levels(daughter, rep, depth=depth+1)
            if parse_avl(daughter) != "syl":
                rep.append(parse_avl(daughter))

    return rep

In [ ]:
def parse_avl(node):
    return node[0][1][1]

def parse_syll(node):
    return {
        "syll": node[0][1][1],
        "dur": node[0][2][1],
        "end":  node[0][3][1]
    }

In [ ]:
nodes_edges_from_hrg(pth)

In [ ]:
res = {}
hrg_paths = glob.glob('/usr0/home/amadaan/data/audio/LJSpeech-1.1/TTS/hrg/*.hrg')
for pth in tqdm(hrg_paths, total=len(hrg_paths)):
    res[basename(pth).split(".")[0]] = nodes_edges_from_hrg(pth)
    

In [ ]:
res['LJ016-0056']

In [ ]:
basepath = "/usr0/home/amadaan/data/audio/LJSpeech-1.1/hrg"

In [ ]:
meta_hrg = "/usr0/home/amadaan/data/audio/LJSpeech-1.1/metadata_hrg_3.csv"

In [ ]:
!head -1 "/usr0/home/amadaan/data/audio/LJSpeech-1.1/metadata_hrg_3.csv"

In [ ]:
with open(meta_hrg, "w") as fout:
    for k, v in res.items():
        write_json(f"{basepath}/{k}.json", v)
        v_str = json.dumps(v)
        fout.write(f"{k}|{v_str}|{v_str}\n")
    

In [ ]:
def write_json(pth, d):
    with open(pth, "w") as f:
        json.dump(d, f)

In [ ]:
LIST = [NODE, LIST]